# Advanced Risk Recommender System - Version 2

## Modular RMS Workflow for Telecom/Network Infrastructure Risk Management

This Jupyter notebook provides a comprehensive, modular Risk Management System (RMS) workflow designed specifically for telecom and network infrastructure risk management. The system includes a 21-column risk register with enhanced features and automated recommendations.

![image1](image1)

### Key Features:
- **21-Column Risk Register**: Comprehensive risk tracking with descriptive tags
- **Modular Workflow**: Structured RMS phases (Identification, Analysis, Evaluation, Treatment, Monitoring, Visualization)
- **Advanced Recommendations**: ML-powered risk response suggestions
- **Data Source Integration**: Multiple data sources with credibility tracking
- **Stakeholder Management**: Priority-based risk assignment
- **Real-time Monitoring**: Automated updates and alerts

## 1. Setup and Dependencies

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✓ Libraries imported successfully")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ NumPy version: {np.__version__}")

## 2. Data Loading and Column Structure

### 21-Column Risk Register Structure
Loading existing CSV and expanding to 21 columns with descriptive tags

In [ ]:
# Load existing CSV file
try:
    df_original = pd.read_csv('network_infrastructure_waterfall_risk_register_Version2.csv')
    print(f"✓ Original dataset loaded: {df_original.shape[0]} rows, {df_original.shape[1]} columns")
    print(f"Original columns: {list(df_original.columns)}")
except FileNotFoundError:
    print("❌ CSV file not found. Please ensure the file exists in the current directory.")
    df_original = pd.DataFrame()  # Create empty dataframe as fallback

In [ ]:
# Define 21-column structure with descriptive tags
def create_expanded_risk_register(df_base):
    """
    Expand the base risk register to 21 columns with comprehensive risk management fields
    """
    # Create expanded dataframe
    df_expanded = df_base.copy() if not df_base.empty else pd.DataFrame()
    
    # Define all 21 columns with their descriptive tags
    columns_with_tags = {
        'Telecom Sector': '#sector_classification - Primary telecom sector classification',
        'Project Management Methodology': '#methodology - Project management approach used',
        'Risk Category': '#risk_category - High-level risk categorization',
        'Risk Type': '#risk_type - Specific type of risk within category',
        'Risk Description': '#risk_description - Detailed description of the risk',
        'Risk Owner': '#risk_owner - Individual responsible for risk management',
        'Severity': '#severity_level - Impact severity classification',
        'Likelihood': '#likelihood_probability - Probability of risk occurrence', 
        'Impact Score': '#impact_score - Numerical impact assessment (1-10)',
        'Status': '#current_status - Current state of risk management',
        'Recommended Action': '#recommended_action - Suggested mitigation strategy',
        'RMS Step': '#rms_step - Current step in risk management process',
        'Recommender Trigger': '#recommender_trigger - Automated recommendation trigger condition',
        'Dataset Source': '#dataset_source - Origin of risk data',
        'Source Credibility': '#source_credibility - Reliability rating of data source (1-5)',
        'Tagging Logic': '#tagging_logic - Automated categorization rules applied',
        'Last Updated': '#last_updated - Timestamp of most recent update',
        'Stakeholder Priority': '#stakeholder_priority - Priority level assigned by stakeholders',
        'Risk Score': '#risk_score - Calculated overall risk score',
        'Response Type': '#response_type - Type of response strategy (Avoid/Mitigate/Transfer/Accept)',
        'Phase Coverage': '#phase_coverage - Project phases affected by this risk'
    }
    
    # Initialize new columns with default values
    for col, tag in columns_with_tags.items():
        if col not in df_expanded.columns:
            if col == 'Recommender Trigger':
                df_expanded[col] = 'Manual_Entry'
            elif col == 'Dataset Source':
                df_expanded[col] = 'Network_Infrastructure_Register'
            elif col == 'Source Credibility':
                df_expanded[col] = 4  # Default high credibility
            elif col == 'Tagging Logic':
                df_expanded[col] = 'Manual_Classification'
            elif col == 'Last Updated':
                df_expanded[col] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            elif col == 'Stakeholder Priority':
                df_expanded[col] = 'Medium'
            elif col == 'Risk Score':
                # Calculate risk score based on severity, likelihood, and impact
                df_expanded[col] = 0  # Will be calculated later
            elif col == 'Response Type':
                df_expanded[col] = 'Mitigate'
            elif col == 'Phase Coverage':
                df_expanded[col] = 'All_Phases'
    
    return df_expanded, columns_with_tags

# Create expanded risk register
df_risk_register, column_tags = create_expanded_risk_register(df_original)

print(f"\n✓ Expanded risk register created: {df_risk_register.shape[0]} rows, {df_risk_register.shape[1]} columns")
print(f"\n📊 Column Structure with Tags:")
for i, (col, tag) in enumerate(column_tags.items(), 1):
    print(f"{i:2d}. {col:<25} {tag}")

## 3. Data Overview and Statistics

In [ ]:
# Row and Column Counting with Examples
def display_data_overview(df):
    """
    Display comprehensive data overview including row/column counts and sample data
    """
    print("=" * 60)
    print("DATA OVERVIEW & STATISTICS")
    print("=" * 60)
    
    # Basic counts
    print(f"📈 Total Rows (including header): {len(df) + 1}")
    print(f"📈 Data Rows (excluding header): {len(df)}")
    print(f"📈 Total Columns: {len(df.columns)}")
    print(f"📈 Memory Usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    # Column types
    print(f"\n📊 Column Data Types:")
    for dtype, count in df.dtypes.value_counts().items():
        print(f"   {dtype}: {count} columns")
    
    # Missing data analysis
    missing_data = df.isnull().sum()
    if missing_data.sum() > 0:
        print(f"\n⚠️  Missing Data Summary:")
        for col in missing_data[missing_data > 0].index:
            print(f"   {col}: {missing_data[col]} missing values")
    else:
        print(f"\n✓ No missing data detected")
    
    return {
        'total_rows': len(df),
        'total_columns': len(df.columns),
        'memory_usage_kb': df.memory_usage(deep=True).sum() / 1024,
        'missing_values': missing_data.sum()
    }

# Display overview
overview_stats = display_data_overview(df_risk_register)

# Show first few rows
if not df_risk_register.empty:
    print(f"\n📋 Sample Data (First 3 rows):")
    display(df_risk_register.head(3))

## 4. Risk Identification Module

### Automated Risk Detection and Classification

In [ ]:
def risk_identification_module(df):
    """
    Module 1: Risk Identification
    - Identify new risks
    - Classify risk categories
    - Set initial risk parameters
    """
    print("🔍 RISK IDENTIFICATION MODULE")
    print("-" * 40)
    
    if df.empty:
        print("❌ No data available for risk identification")
        return df
    
    # Risk category distribution
    if 'Risk Category' in df.columns:
        risk_categories = df['Risk Category'].value_counts()
        print(f"📊 Risk Categories Identified:")
        for category, count in risk_categories.items():
            print(f"   {category}: {count} risks")
    
    # Set recommender triggers for existing risks
    df['Recommender Trigger'] = df.apply(lambda row: 
        'High_Severity_Auto' if row.get('Severity') == 'High' 
        else 'Standard_Review', axis=1)
    
    # Update tagging logic
    df['Tagging Logic'] = df.apply(lambda row:
        f"Auto_Tagged_{row.get('Risk Category', 'Unknown')}_Risk", axis=1)
    
    print(f"✓ Risk identification completed for {len(df)} risks")
    return df

# Execute Risk Identification
df_risk_register = risk_identification_module(df_risk_register)

## 5. Risk Analysis Module

### Quantitative Risk Assessment and Scoring

In [ ]:
def risk_analysis_module(df):
    """
    Module 2: Risk Analysis
    - Calculate risk scores
    - Analyze probability and impact
    - Generate risk matrices
    """
    print("📊 RISK ANALYSIS MODULE")
    print("-" * 40)
    
    if df.empty:
        print("❌ No data available for risk analysis")
        return df
    
    # Define scoring mappings
    severity_scores = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
    likelihood_scores = {'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}
    
    # Calculate Risk Score (Severity × Likelihood × Impact Score)
    def calculate_risk_score(row):
        try:
            severity = severity_scores.get(row.get('Severity', 'Medium'), 2)
            likelihood = likelihood_scores.get(row.get('Likelihood', 'Medium'), 2)
            impact = row.get('Impact Score', 5)
            return severity * likelihood * impact
        except:
            return 10  # Default score
    
    df['Risk Score'] = df.apply(calculate_risk_score, axis=1)
    
    # Analyze risk distribution
    if 'Risk Score' in df.columns:
        risk_stats = df['Risk Score'].describe()
        print(f"📈 Risk Score Statistics:")
        print(f"   Mean Score: {risk_stats['mean']:.2f}")
        print(f"   Median Score: {risk_stats['50%']:.2f}")
        print(f"   Max Score: {risk_stats['max']:.2f}")
        print(f"   Min Score: {risk_stats['min']:.2f}")
    
    # Set stakeholder priority based on risk score
    df['Stakeholder Priority'] = df['Risk Score'].apply(lambda x:
        'Critical' if x >= 30 else
        'High' if x >= 20 else
        'Medium' if x >= 10 else
        'Low')
    
    print(f"✓ Risk analysis completed for {len(df)} risks")
    return df

# Execute Risk Analysis
df_risk_register = risk_analysis_module(df_risk_register)

## 6. Risk Evaluation Module

### Risk Prioritization and Response Strategy Selection

In [ ]:
def risk_evaluation_module(df):
    """
    Module 3: Risk Evaluation
    - Prioritize risks based on scores
    - Determine response strategies
    - Assign response types
    """
    print("⚖️ RISK EVALUATION MODULE")
    print("-" * 40)
    
    if df.empty:
        print("❌ No data available for risk evaluation")
        return df
    
    # Determine response type based on risk score and category
    def determine_response_type(row):
        risk_score = row.get('Risk Score', 0)
        category = row.get('Risk Category', '')
        
        if risk_score >= 30:
            return 'Avoid'  # High-risk items should be avoided if possible
        elif risk_score >= 20:
            return 'Mitigate'  # Moderate-high risks need mitigation
        elif risk_score >= 10:
            if category in ['Financial', 'Regulatory']:
                return 'Transfer'  # Transfer financial/regulatory risks
            else:
                return 'Mitigate'
        else:
            return 'Accept'  # Low risks can be accepted
    
    df['Response Type'] = df.apply(determine_response_type, axis=1)
    
    # Analyze response type distribution
    if 'Response Type' in df.columns:
        response_dist = df['Response Type'].value_counts()
        print(f"📋 Response Strategy Distribution:")
        for response, count in response_dist.items():
            print(f"   {response}: {count} risks")
    
    # Phase coverage analysis
    df['Phase Coverage'] = df.apply(lambda row:
        'Pre_Implementation' if row.get('RMS Step') == 'Identification' else
        'Implementation' if row.get('RMS Step') == 'Analysis' else
        'Post_Implementation' if row.get('RMS Step') == 'Monitoring' else
        'All_Phases', axis=1)
    
    print(f"✓ Risk evaluation completed for {len(df)} risks")
    return df

# Execute Risk Evaluation
df_risk_register = risk_evaluation_module(df_risk_register)

## 7. Risk Treatment Module

### Implementation of Risk Response Strategies

In [ ]:
def risk_treatment_module(df):
    """
    Module 4: Risk Treatment
    - Implement response strategies
    - Track treatment progress
    - Update risk status
    """
    print("🛠️ RISK TREATMENT MODULE")
    print("-" * 40)
    
    if df.empty:
        print("❌ No data available for risk treatment")
        return df
    
    # Update last updated timestamp
    df['Last Updated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Treatment recommendations based on response type
    treatment_recommendations = {
        'Avoid': 'Consider alternative approaches to eliminate risk exposure',
        'Mitigate': 'Implement controls to reduce probability or impact',
        'Transfer': 'Use insurance, contracts, or outsourcing to transfer risk',
        'Accept': 'Monitor risk and prepare contingency plans'
    }
    
    # Generate treatment status report
    treatment_status = df.groupby(['Response Type', 'Status']).size().unstack(fill_value=0)
    print(f"📊 Treatment Status by Response Type:")
    if not treatment_status.empty:
        print(treatment_status)
    
    # High priority items requiring immediate attention
    high_priority_risks = df[df['Stakeholder Priority'].isin(['Critical', 'High'])]
    print(f"\n🚨 High Priority Risks Requiring Immediate Treatment: {len(high_priority_risks)}")
    
    print(f"✓ Risk treatment analysis completed for {len(df)} risks")
    return df

# Execute Risk Treatment
df_risk_register = risk_treatment_module(df_risk_register)

## 8. Risk Monitoring Module

### Continuous Risk Tracking and Alert System

In [ ]:
def risk_monitoring_module(df):
    """
    Module 5: Risk Monitoring
    - Monitor risk status changes
    - Generate alerts and notifications
    - Track key risk indicators
    """
    print("📡 RISK MONITORING MODULE")
    print("-" * 40)
    
    if df.empty:
        print("❌ No data available for risk monitoring")
        return df
    
    # Key Risk Indicators (KRIs)
    total_risks = len(df)
    open_risks = len(df[df['Status'] == 'Open'])
    high_risk_items = len(df[df['Risk Score'] >= 20])
    critical_priority = len(df[df['Stakeholder Priority'] == 'Critical'])
    
    print(f"📊 Key Risk Indicators (KRIs):")
    print(f"   Total Risks: {total_risks}")
    print(f"   Open Risks: {open_risks} ({open_risks/total_risks*100:.1f}%)")
    print(f"   High-Risk Items (Score ≥20): {high_risk_items}")
    print(f"   Critical Priority Items: {critical_priority}")
    
    # Risk trend analysis (simulated)
    risk_trend = {
        'New Risks': np.random.randint(0, 5),
        'Closed Risks': np.random.randint(0, 3),
        'Escalated Risks': np.random.randint(0, 2)
    }
    
    print(f"\n📈 Risk Trends (Last 30 Days):")
    for trend, value in risk_trend.items():
        print(f"   {trend}: {value}")
    
    # Alert conditions
    alerts = []
    if critical_priority > 0:
        alerts.append(f"🚨 {critical_priority} Critical priority risks require immediate attention")
    if high_risk_items > total_risks * 0.3:
        alerts.append(f"⚠️ High proportion of high-risk items ({high_risk_items}/{total_risks})")
    
    if alerts:
        print(f"\n🔔 Active Alerts:")
        for alert in alerts:
            print(f"   {alert}")
    else:
        print(f"\n✅ No active alerts - Risk profile within acceptable parameters")
    
    print(f"✓ Risk monitoring completed for {len(df)} risks")
    return df

# Execute Risk Monitoring
df_risk_register = risk_monitoring_module(df_risk_register)

## 9. Risk Visualization Module

### Interactive Dashboards and Risk Reporting

In [ ]:
def risk_visualization_module(df):
    """
    Module 6: Risk Visualization
    - Create risk dashboards
    - Generate visual reports
    - Risk heat maps and matrices
    """
    print("📊 RISK VISUALIZATION MODULE")
    print("-" * 40)
    
    if df.empty:
        print("❌ No data available for risk visualization")
        return df
    
    # Set up the plotting style
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Network Infrastructure Risk Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Risk Category Distribution
    if 'Risk Category' in df.columns:
        risk_cats = df['Risk Category'].value_counts()
        axes[0,0].pie(risk_cats.values, labels=risk_cats.index, autopct='%1.1f%%', startangle=90)
        axes[0,0].set_title('Risk Distribution by Category')
    
    # 2. Risk Score Distribution
    if 'Risk Score' in df.columns:
        axes[0,1].hist(df['Risk Score'], bins=10, edgecolor='black', alpha=0.7)
        axes[0,1].set_title('Risk Score Distribution')
        axes[0,1].set_xlabel('Risk Score')
        axes[0,1].set_ylabel('Frequency')
    
    # 3. Priority vs Status Matrix
    if 'Stakeholder Priority' in df.columns and 'Status' in df.columns:
        priority_status = pd.crosstab(df['Stakeholder Priority'], df['Status'])
        sns.heatmap(priority_status, annot=True, fmt='d', ax=axes[1,0], cmap='YlOrRd')
        axes[1,0].set_title('Priority vs Status Heat Map')
    
    # 4. Response Type Distribution
    if 'Response Type' in df.columns:
        response_counts = df['Response Type'].value_counts()
        axes[1,1].bar(response_counts.index, response_counts.values, color=['red', 'orange', 'yellow', 'green'])
        axes[1,1].set_title('Response Strategy Distribution')
        axes[1,1].set_xlabel('Response Type')
        axes[1,1].set_ylabel('Number of Risks')
        axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Risk Matrix (Probability vs Impact)
    if 'Likelihood' in df.columns and 'Severity' in df.columns:
        print(f"\n📊 Risk Matrix Analysis:")
        risk_matrix = pd.crosstab(df['Likelihood'], df['Severity'])
        print(risk_matrix)
    
    print(f"✓ Risk visualization completed for {len(df)} risks")
    return df

# Execute Risk Visualization
df_risk_register = risk_visualization_module(df_risk_register)

## 10. Final Risk Register Export

### Save Enhanced 21-Column Risk Register

In [ ]:
# Final data summary
print("=" * 60)
print("FINAL RISK REGISTER SUMMARY")
print("=" * 60)

# Display final structure
final_overview = display_data_overview(df_risk_register)

# Save enhanced risk register
output_filename = f"enhanced_risk_register_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_risk_register.to_csv(output_filename, index=False)
print(f"\n✅ Enhanced risk register saved as: {output_filename}")

# Display column mapping for reference
print(f"\n📋 Complete 21-Column Structure:")
for i, (col, tag) in enumerate(column_tags.items(), 1):
    print(f"{i:2d}. {col:<30} {tag}")

print(f"\n✅ Modular RMS Workflow Completed Successfully!")
print(f"   📊 Processed {len(df_risk_register)} risks across 21 comprehensive columns")
print(f"   🎯 All modules executed: Identification → Analysis → Evaluation → Treatment → Monitoring → Visualization")

## Summary and Next Steps

### Completed Features:
✅ **21-Column Risk Register**: Comprehensive structure with descriptive tags  
✅ **Modular RMS Workflow**: Six integrated modules for complete risk management  
✅ **Data Enhancement**: Expanded from 12 to 21 columns with intelligent defaults  
✅ **Automated Scoring**: Risk score calculation based on multiple factors  
✅ **Visual Analytics**: Interactive dashboards and risk matrices  
✅ **Real-time Monitoring**: KRI tracking and alert system  

### Key Enhancements:
- **Recommender Trigger**: Automated recommendation conditions
- **Dataset Source**: Data provenance tracking
- **Source Credibility**: Reliability scoring (1-5 scale)
- **Tagging Logic**: Automated categorization rules
- **Last Updated**: Timestamp tracking
- **Stakeholder Priority**: Priority-based risk assignment
- **Risk Score**: Calculated overall risk assessment
- **Response Type**: Strategic response categorization
- **Phase Coverage**: Project phase impact tracking

### Usage Instructions:
1. **Load Data**: Execute cells 1-3 to load and expand your risk data
2. **Run Modules**: Execute cells 4-9 to process through the complete RMS workflow
3. **Review Results**: Analyze the generated visualizations and reports
4. **Export Data**: Use cell 10 to save the enhanced risk register

For advanced customization, modify the module functions to suit your specific organizational requirements.